In [1]:
import numpy as np
import pandas as pd
import time
import gensim
import re
from gensim.models.word2vec import Word2Vec 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('../adam/datasets/twitter_scrape_5.10.csv')

pd.set_option('display.max_colwidth', -1)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start
0,1230271896752730117,Paloma Escudero,Coronavirus disease (COVID-19): What parents should know @unicef https://shar.es/a3MeiZ #COVID19 #coronavirus #parenting,2020-02-19 23:24:12,COVID,"{'city': 'Brooklyn', 'lat': 40.650002, 'long': -73.949997, 'radius': '10mi', 'queries': ['COVID', 'COVID-19', 'Corona', 'Coronavirus', 'Quarantine', '#COVID', '#COVID-19', '#quarantine', '#Quarantine', '#covid19'], 'start_year': 2020, 'start_month': 2, 'start_day': 1}",40.650002,-73.949997,10mi,2020-02-01
1,1230192345087447040,RLC Student Media,"More flu deaths in Dallas, COVID-19 now in Texas http://www.richlandstudentmedia.com/chronicle-posts/2020/2/19/more-flu-deaths-in-dallas-covid-19-now-in-texas …",2020-02-19 18:08:05,COVID,"{'city': 'Brooklyn', 'lat': 40.650002, 'long': -73.949997, 'radius': '10mi', 'queries': ['COVID', 'COVID-19', 'Corona', 'Coronavirus', 'Quarantine', '#COVID', '#COVID-19', '#quarantine', '#Quarantine', '#covid19'], 'start_year': 2020, 'start_month': 2, 'start_day': 1}",40.650002,-73.949997,10mi,2020-02-01
2,1230096130748317696,Dr Janette Nesheiwat,Flu kills. Worst season I’ve seen in years. I worry about the flu not covid-19,2020-02-19 11:45:46,COVID,"{'city': 'Brooklyn', 'lat': 40.650002, 'long': -73.949997, 'radius': '10mi', 'queries': ['COVID', 'COVID-19', 'Corona', 'Coronavirus', 'Quarantine', '#COVID', '#COVID-19', '#quarantine', '#Quarantine', '#covid19'], 'start_year': 2020, 'start_month': 2, 'start_day': 1}",40.650002,-73.949997,10mi,2020-02-01
3,1229810497341599744,Element Healing Arts,"Earlier this week, Bloomberg News reported that 2,200 Traditional Chinese Medicine (TCM) workers were sent to Hubei province, which is at the center of the COVID-19 outbreak. In the Western medical establishment,… https://www.instagram.com/p/B8tynW0AwoR/?igshid=1oge1iig15xa3 …",2020-02-18 16:50:45,COVID,"{'city': 'Brooklyn', 'lat': 40.650002, 'long': -73.949997, 'radius': '10mi', 'queries': ['COVID', 'COVID-19', 'Corona', 'Coronavirus', 'Quarantine', '#COVID', '#COVID-19', '#quarantine', '#Quarantine', '#covid19'], 'start_year': 2020, 'start_month': 2, 'start_day': 1}",40.650002,-73.949997,10mi,2020-02-01
4,1229567925004242944,Darwish,Chinese scientists says COVID-19/coronavirus could have originated from ... https://youtu.be/ZC0gww2yznI via @YouTube,2020-02-18 00:46:52,COVID,"{'city': 'Brooklyn', 'lat': 40.650002, 'long': -73.949997, 'radius': '10mi', 'queries': ['COVID', 'COVID-19', 'Corona', 'Coronavirus', 'Quarantine', '#COVID', '#COVID-19', '#quarantine', '#Quarantine', '#covid19'], 'start_year': 2020, 'start_month': 2, 'start_day': 1}",40.650002,-73.949997,10mi,2020-02-01


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(49536, 10)

In [5]:
df['token_text'] = df['text'].str.replace('([^ a-zA-Z0-9])', '')
df['split_text'] = df['token_text'].astype(str).str.lower().str.split()

In [6]:
sent = [row for row in df['split_text']]

In [7]:
sent

[['coronavirus',
  'disease',
  'covid19',
  'what',
  'parents',
  'should',
  'know',
  'unicef',
  'httpssharesa3meiz',
  'covid19',
  'coronavirus',
  'parenting'],
 ['more',
  'flu',
  'deaths',
  'in',
  'dallas',
  'covid19',
  'now',
  'in',
  'texas',
  'httpwwwrichlandstudentmediacomchronicleposts2020219morefludeathsindallascovid19nowintexas'],
 ['flu',
  'kills',
  'worst',
  'season',
  'ive',
  'seen',
  'in',
  'years',
  'i',
  'worry',
  'about',
  'the',
  'flu',
  'not',
  'covid19'],
 ['earlier',
  'this',
  'week',
  'bloomberg',
  'news',
  'reported',
  'that',
  '2200',
  'traditional',
  'chinese',
  'medicine',
  'tcm',
  'workers',
  'were',
  'sent',
  'to',
  'hubei',
  'province',
  'which',
  'is',
  'at',
  'the',
  'center',
  'of',
  'the',
  'covid19',
  'outbreak',
  'in',
  'the',
  'western',
  'medical',
  'establishment',
  'httpswwwinstagramcompb8tynw0aworigshid1oge1iig15xa3'],
 ['chinese',
  'scientists',
  'says',
  'covid19coronavirus',
  'cou

In [8]:
tweet_w2v = Word2Vec(size=200, min_count=10)

In [9]:
tweet_w2v.build_vocab(sent)

In [10]:
tweet_w2v.wv.vocab

{'coronavirus': <gensim.models.keyedvectors.Vocab at 0x1a268809d0>,
 'disease': <gensim.models.keyedvectors.Vocab at 0x1a26880990>,
 'covid19': <gensim.models.keyedvectors.Vocab at 0x1a26880a90>,
 'what': <gensim.models.keyedvectors.Vocab at 0x1a2e836850>,
 'parents': <gensim.models.keyedvectors.Vocab at 0x1a2e8360d0>,
 'should': <gensim.models.keyedvectors.Vocab at 0x1a2e836ad0>,
 'know': <gensim.models.keyedvectors.Vocab at 0x1a2e836050>,
 'unicef': <gensim.models.keyedvectors.Vocab at 0x1a2e836290>,
 'httpssharesa3meiz': <gensim.models.keyedvectors.Vocab at 0x1a2e836a50>,
 'parenting': <gensim.models.keyedvectors.Vocab at 0x1a2e8361d0>,
 'more': <gensim.models.keyedvectors.Vocab at 0x1a2e836810>,
 'flu': <gensim.models.keyedvectors.Vocab at 0x1a2e836710>,
 'deaths': <gensim.models.keyedvectors.Vocab at 0x1a2e836490>,
 'in': <gensim.models.keyedvectors.Vocab at 0x1a2e836590>,
 'dallas': <gensim.models.keyedvectors.Vocab at 0x1a2e836150>,
 'now': <gensim.models.keyedvectors.Vocab at 0

In [11]:
tweet_w2v.most_similar('covid19')

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('protective', 0.25442397594451904),
 ('gear', 0.23566126823425293),
 ('dick', 0.22984835505485535),
 ('egypt', 0.22632697224617004),
 ('nightphotography', 0.2217760682106018),
 ('sentences', 0.21838253736495972),
 ('transblood', 0.21572613716125488),
 ('kindness', 0.2147969752550125),
 ('writing', 0.21398811042308807),
 ('coherence', 0.21269044280052185)]